In [16]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import ClippedAdam
import numpy as np
import pandas as pd
import glob, os
from sklearn.metrics import mean_squared_error

In [17]:
# --- Load and prepare data ---
def load_data(data_dir, seq_len=18):
    paths = sorted(
        glob.glob(os.path.join(data_dir, "IPF_Final_*.csv")),
        key=lambda p: int(os.path.basename(p).split('_')[2].split('.')[0])
    )
    assert len(paths) >= seq_len + 1, f"Need at least {seq_len + 1} CSV files for full sequence + target."

    arrays = [np.loadtxt(path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) for path in paths[:seq_len + 1]]
    X_seq = np.stack(arrays[:seq_len], axis=0)
    Y_arr = arrays[seq_len]

    x_tensor = torch.tensor(X_seq.transpose(1, 0, 2), dtype=torch.float32)
    y_tensor = torch.tensor(Y_arr, dtype=torch.float32)
    return x_tensor, y_tensor

data_dir = "/kaggle/input/training/Training Dataset"
seq_len = 18
x_full, y_full = load_data(data_dir, seq_len)
x_full = x_full.to(device)
y_full = y_full.to(device)


In [26]:
class MultiLSTMBackbone(nn.Module):
    def __init__(self, input_size=3, hidden_size=64):
        super().__init__()
        self.lstm_phi1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm_phi = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm_phi2 = nn.LSTM(input_size, hidden_size, batch_first=True)

        self.fc_phi1 = nn.Linear(hidden_size * 3, 1)
        self.fc_phi = nn.Linear(hidden_size * 3, 1)
        self.fc_phi2 = nn.Linear(hidden_size * 3, 1)

    def forward(self, x):
        _, (h1, _) = self.lstm_phi1(x)
        _, (h2, _) = self.lstm_phi(x)
        _, (h3, _) = self.lstm_phi2(x)
        h_cat = torch.cat([h1[-1], h2[-1], h3[-1]], dim=1)
        out1 = self.fc_phi1(h_cat)
        out2 = self.fc_phi(h_cat)
        out3 = self.fc_phi2(h_cat)
        return out1, out2, out3

In [27]:
def model_fn(x, y=None):
    y1, y2, y3 = (y[:, 0:1], y[:, 1:2], y[:, 2:3]) if y is not None else (None, None, None)

    _, (h1, _) = model.lstm_phi1(x)
    _, (h2, _) = model.lstm_phi(x)
    _, (h3, _) = model.lstm_phi2(x)
    h_combined = torch.cat([h1[-1], h2[-1], h3[-1]], dim=1)

    w1 = pyro.sample("w1", dist.Normal(model.fc_phi1.weight, 0.5).to_event(2))
    b1 = pyro.sample("b1", dist.Normal(model.fc_phi1.bias, 0.5).to_event(1))
    w2 = pyro.sample("w2", dist.Normal(model.fc_phi.weight, 0.5).to_event(2))
    b2 = pyro.sample("b2", dist.Normal(model.fc_phi.bias, 0.5).to_event(1))
    w3 = pyro.sample("w3", dist.Normal(model.fc_phi2.weight, 0.5).to_event(2))
    b3 = pyro.sample("b3", dist.Normal(model.fc_phi2.bias, 0.5).to_event(1))

    mean1 = h_combined @ w1.t() + b1
    mean2 = h_combined @ w2.t() + b2
    mean3 = h_combined @ w3.t() + b3

    with pyro.plate("data", x.shape[0]):
        pyro.sample("obs1", dist.Normal(mean1, 5.0).to_event(1), obs=y1)
        pyro.sample("obs2", dist.Normal(mean2, 5.0).to_event(1), obs=y2)
        pyro.sample("obs3", dist.Normal(mean3, 5.0).to_event(1), obs=y3)

In [28]:
def guide_fn(x, y=None):
    for i, head in enumerate([model.fc_phi1, model.fc_phi, model.fc_phi2], start=1):
        pyro.param(f"w{i}_loc", head.weight.detach().clone())
        pyro.param(f"w{i}_scale", torch.ones_like(head.weight) * 0.5, constraint=dist.constraints.positive)
        pyro.param(f"b{i}_loc", head.bias.detach().clone())
        pyro.param(f"b{i}_scale", torch.ones_like(head.bias) * 0.5, constraint=dist.constraints.positive)

        pyro.sample(f"w{i}", dist.Normal(pyro.param(f"w{i}_loc"), pyro.param(f"w{i}_scale")).to_event(2))
        pyro.sample(f"b{i}", dist.Normal(pyro.param(f"b{i}_loc"), pyro.param(f"b{i}_scale")).to_event(1))


# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiLSTMBackbone().to(device)

In [30]:
# --- LSTM Backbone Pretraining ---
print("🔧 Pretraining deterministic LSTM backbone...")
backbone_optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

model.train()
for epoch in range(2000):
    backbone_optimizer.zero_grad()
    out1, out2, out3 = model(x_full)
    loss = (
        0.33 * loss_fn(out1, y_full[:, 0:1]) +
        0.33 * loss_fn(out2, y_full[:, 1:2]) +
        0.34 * loss_fn(out3, y_full[:, 2:3])
    )
    loss.backward()
    backbone_optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1} LSTM Backbone Loss: {loss:.2f}")

🔧 Pretraining deterministic LSTM backbone...
Epoch 100 LSTM Backbone Loss: 2092.07
Epoch 200 LSTM Backbone Loss: 1832.75
Epoch 300 LSTM Backbone Loss: 1595.78
Epoch 400 LSTM Backbone Loss: 1381.92
Epoch 500 LSTM Backbone Loss: 1195.73
Epoch 600 LSTM Backbone Loss: 1035.11
Epoch 700 LSTM Backbone Loss: 896.31
Epoch 800 LSTM Backbone Loss: 780.35
Epoch 900 LSTM Backbone Loss: 683.00
Epoch 1000 LSTM Backbone Loss: 600.45
Epoch 1100 LSTM Backbone Loss: 530.93
Epoch 1200 LSTM Backbone Loss: 474.01
Epoch 1300 LSTM Backbone Loss: 424.84
Epoch 1400 LSTM Backbone Loss: 384.92
Epoch 1500 LSTM Backbone Loss: 349.46
Epoch 1600 LSTM Backbone Loss: 320.50
Epoch 1700 LSTM Backbone Loss: 293.34
Epoch 1800 LSTM Backbone Loss: 269.06
Epoch 1900 LSTM Backbone Loss: 249.05
Epoch 2000 LSTM Backbone Loss: 229.61


In [31]:
# --- VI Training ---
pyro.clear_param_store()
optimizer = ClippedAdam({"lr": 1e-3})
svi = SVI(model_fn, guide_fn, optimizer, loss=Trace_ELBO())

# Run SVI for multiple steps with early stopping
print("🔁 Running Variational Inference with early stopping...")
num_steps = 2000
patience = 100
min_delta = 1.0
best_loss = float('inf')
no_improve_count = 0

for step in range(num_steps):
    loss = svi.step(x_full, y_full)

    if loss < best_loss - min_delta:
        best_loss = loss
        no_improve_count = 0
    else:
        no_improve_count += 1

    if (step + 1) % 100 == 0:
        print(f"Step {step + 1} 	ELBO Loss: {loss:.2f}")

    if no_improve_count >= patience:
        print(f"⏹️ Early stopping at step {step + 1} due to no improvement.")
        break


🔁 Running Variational Inference with early stopping...
Step 100 	ELBO Loss: 211556.23
Step 200 	ELBO Loss: 215390.72
Step 300 	ELBO Loss: 211705.99
Step 400 	ELBO Loss: 202261.13
Step 500 	ELBO Loss: 189673.01
Step 600 	ELBO Loss: 193629.38
⏹️ Early stopping at step 631 due to no improvement.


In [32]:
# --- Inference using variational posterior mean ---
model.eval()
with torch.no_grad():
    _, (h1, _) = model.lstm_phi1(x_full)
    _, (h2, _) = model.lstm_phi(x_full)
    _, (h3, _) = model.lstm_phi2(x_full)
    h_combined = torch.cat([h1[-1], h2[-1], h3[-1]], dim=1)

    phi1 = h_combined @ pyro.param("w1_loc").t() + pyro.param("b1_loc")
    phi  = h_combined @ pyro.param("w2_loc").t() + pyro.param("b2_loc")
    phi2 = h_combined @ pyro.param("w3_loc").t() + pyro.param("b3_loc")

    preds = torch.cat([phi1, phi, phi2], dim=1).cpu().numpy()


In [34]:
# Save prediction and compute RMSE
y_true_np = y_full.cpu().numpy()
rmse = np.sqrt(np.mean((preds - y_true_np) ** 2))
print(f"✅ Final VI RMSE: {rmse:.4f} degrees")

df = pd.DataFrame(preds, columns=["Phi1", "Phi", "Phi2"])
Phase = [1 for i in range(len(df))]
df["Phase"] = Phase
df = df[["Phase", "Phi1", "Phi", "Phi2"]]
df.to_csv("IPF_20_pred.csv", index=False)
print("✅ Prediction for 20th timestep saved to IPF_20_pred.csv")

✅ Final VI RMSE: 13.3633 degrees
✅ Prediction for 20th timestep saved to IPF_20_pred.csv


In [ ]:
# This is the RMSE for model training
# For model evaluation, predict the 21% strained microstructure and compare with actual data.